# Backtester is WIP

# Setup and load

In [74]:
import pandas as pd
import backtrader as bt
import datetime
import os
import sys
import math

# Load the data into a Pandas DataFrame

# Set the path to the root directory
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(path)

# Load Model 3 prediction file
df = pd.read_csv(path + '/data/predictions/12.csv')

df['Expiry_date'] = pd.to_datetime(df['Quote_date']) + pd.to_timedelta(df['TTM'], unit='D')

In [75]:
# Filter < 2018-01-01
df = df[df['Quote_date'] <= '2018-01-01']

In [76]:
display(df)

,Unnamed: 0,Quote_date,Price,Prediction,Underlying_last,Strike,TTM,R,Expiry_date
0,3134913,2017-12-22,1083.045,1083.672000,2683.72,1600.0,4,1.15,2017-12-26
1,3134914,2017-12-22,983.050,983.439100,2683.72,1700.0,4,1.15,2017-12-26
2,3134915,2017-12-22,933.045,933.258100,2683.72,1750.0,4,1.15,2017-12-26
3,3134916,2017-12-22,883.050,883.086500,2683.72,1800.0,4,1.15,2017-12-26
4,3134917,2017-12-22,833.050,832.966430,2683.72,1850.0,4,1.15,2017-12-26
...,...,...,...,...,...,...,...,...,...
19929,3154842,2017-12-29,107.450,97.967480,2675.22,2950.0,721,1.89,2019-12-20
19930,3154843,2017-12-29,88.800,79.946070,2675.22,3000.0,721,1.89,2019-12-20
19931,3154844,2017-12-29,58.650,51.458393,2675.22,3100.0,721,1.89,2019-12-20
19932,3154845,2017-12-29,37.050,37.055058,2675.22,3200.0,721,1.89,2019-12-20


# Main

In [77]:
last_date = df['Quote_date'].max()
df = df[df['Expiry_date'] < last_date]

# Preprocess the data
df['Quote_date'] = pd.to_datetime(df['Quote_date'])
df.set_index('Quote_date', inplace=True)
df.sort_index(inplace=True)


display(df)

class CustomPandasData(bt.feeds.PandasData):
    lines = ('Prediction', 'Price', 'TTM', 'Strike', 'Underlying_last')
    params = (
        ('Prediction', -1),
        ('Price', -1),
        ('TTM', -1),
        ('Strike', -1),
        ('Underlying_last', -1),
    )



class OptionStrategy(bt.Strategy):

    def __init__(self):
        self.buy_threshold = 8

    def next(self):
        current_date = self.datetime.date()
        current_timestamp = pd.Timestamp(current_date)
        print(f"Current date: {current_date}")

        if current_timestamp in df.index:
            for i, row in df.loc[current_timestamp].iterrows():
                price_difference = row['Prediction'] - row['Price']
                if price_difference >= self.buy_threshold:
                    print("Balance before trade: ", self.broker.getvalue())
                    print("Cash before trade: ", self.broker.get_cash())
                    if self.broker.get_cash() >= row['Price']:
                        print(f'Buying on {current_date} at {row["Price"]}')
                        self.buy(size=1, exectype=bt.Order.Market, price=row['Price'])
                        expire_date = row['Expiry_date']
                        intrinsic_value = max(row['Underlying_last'] - row['Strike'], 0)
                        if not math.isnan(intrinsic_value):
                            self.sell(size=1, exectype=bt.Order.Market, valid=expire_date, price=intrinsic_value)
                        else:
                            print(f'Intrinsic value is NaN for the option on {current_date}')
                    else:
                        print(f'Insufficient cash to buy on {current_date} at {row["Price"]}')
                #else:
                    #print(f'Buy threshold not met on {current_date}: price_difference={price_difference}')
            print("Balance after trade: ", self.broker.getvalue())
            print("Cash after trade: ", self.broker.get_cash())






# Create the data feed
data_feed = CustomPandasData(dataname=df)

# Initialize the backtesting engine
cerebro = bt.Cerebro()

print(cerebro.broker.getvalue())
# Add the custom strategy

cerebro.addstrategy(OptionStrategy)
# Add the data feed
cerebro.adddata(data_feed)

# Set the initial balance
cerebro.broker.set_cash(10000000)

print(cerebro.broker.getvalue())

# Add a commission to account for transaction costs
cerebro.broker.setcommission(commission=0.002)

# Run the backtest
results = cerebro.run()

print("Balance: ", cerebro.broker.getvalue())

if not results:
    print("No trades executed.")
else:
    # Calculate performance metrics
    ending_balance = results[0].broker.get_value()
    print(results)

    print(f'Ending balance: {ending_balance}')


,Unnamed: 0,Price,Prediction,Underlying_last,Strike,TTM,R,Expiry_date
Quote_date,,,,,,,,
2017-12-22,3134913,1083.045,1083.67200,2683.72,1600.0,4,1.15,2017-12-26
2017-12-22,3134914,983.050,983.43910,2683.72,1700.0,4,1.15,2017-12-26
2017-12-22,3134915,933.045,933.25810,2683.72,1750.0,4,1.15,2017-12-26
2017-12-22,3134916,883.050,883.08650,2683.72,1800.0,4,1.15,2017-12-26
2017-12-22,3134917,833.050,832.96643,2683.72,1850.0,4,1.15,2017-12-26
...,...,...,...,...,...,...,...,...
2017-12-26,3139176,0.025,0.00000,2680.42,2840.0,1,1.24,2017-12-27
2017-12-26,3139177,0.025,0.00000,2680.42,2850.0,1,1.24,2017-12-27
2017-12-26,3139178,0.025,0.00000,2680.42,2900.0,1,1.24,2017-12-27


10000.0
10000000
Current date: 2017-12-22
Balance before trade:  10000000.0
Cash before trade:  10000000.0
Buying on 2017-12-22 at 14.905
Balance before trade:  10000000.0
Cash before trade:  10000000.0
Buying on 2017-12-22 at 10.3
Balance before trade:  10000000.0
Cash before trade:  10000000.0
Buying on 2017-12-22 at 6.25
Balance before trade:  10000000.0
Cash before trade:  10000000.0
Buying on 2017-12-22 at 15.995
Balance before trade:  10000000.0
Cash before trade:  10000000.0
Buying on 2017-12-22 at 11.95
Balance before trade:  10000000.0
Cash before trade:  10000000.0
Buying on 2017-12-22 at 8.15
Balance after trade:  10000000.0
Cash after trade:  10000000.0
Current date: 2017-12-22
Balance before trade:  nan
Cash before trade:  10000000.0
Buying on 2017-12-22 at 14.905
Balance before trade:  nan
Cash before trade:  10000000.0
Buying on 2017-12-22 at 10.3
Balance before trade:  nan
Cash before trade:  10000000.0
Buying on 2017-12-22 at 6.25
Balance before trade:  nan
Cash before